# Journal Table

In [2]:
# Save journal id XML
import urllib.request

url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nlmcatalog&term=currentlyindexed&retmax=10000'
urllib.request.urlretrieve(url, 'journal_ids.xml')

('journal_ids.xml', <http.client.HTTPMessage at 0x10573e748>)

In [3]:
# Get list of ID's
import xml.etree.ElementTree as ET
tree = ET.parse('journal_ids.xml')
root = tree.getroot()
journal_ids = [i.text for i in root.findall('IdList/Id')]

In [5]:
# Construct EFetch URLs and retrieve .txt files
# >200 ids HTTP POST method recommended. Not sure how, so I'll execute EFetch
# in batches
# import math

# base = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?'
# dbid = 'db=nlmcatalog&'

n = 200
# chunks = [journal_ids[i * n:(i + 1) * n] \
#           for i in range(math.ceil(len(journal_ids)/n))] 

# i = 0
# for chunk in chunks:
#     EFetchURL = base+dbid+'id='+','.join(chunk)
#     urllib.request.urlretrieve(EFetchURL, f'journals_{i}.txt')
#     i += 1

In [6]:
# Get file list
import math
j_files = [f'journals_{i}.txt' for i in range(math.ceil(len(journal_ids)/n))]

# Explore file
f = open(j_files[0], 'r').readlines()
[i for i in f]

["1. Author(s): University Health Partners of Hawai'i.\n",
 'Title Abbreviation: Hawaii J Health Soc Welf\n',
 "Title(s): Hawai'i journal of health & social welfare.\n",
 'Publication Start Year: 2019\n',
 'Publication End Year: \n',
 'Frequency: Monthly\n',
 'Country of Publication: United States\n',
 "Publisher: Honolulu, Hawaii : University Health Partners of Hawai'i, [2019]-\n",
 'Description: 1 online resource\n',
 'Language: English\n',
 'ISSN: 2641-5224(Electronic); 2641-5216(Print); 2641-5216(Linking)\n',
 'LCCN: 2019201558\n',
 'Electronic Links: , http://hjmph.org, , https://www.ncbi.nlm.nih.gov/pmc/journals/3710/\n',
 'Fully Indexed In: MEDLINE: v78n7, July 2019-\n',
 'In: PubMed: v78n7, July 2019-, PMC; \n',
 'Current Indexing Status: Currently indexed for MEDLINE. \n',
 'Current Subset: Index Medicus\n',
 'Version Indexed: Electronic\n',
 'Notes: Also issued in print.\n',
 'Other ID: (OCoLC)1083524408\n',
 'NLM ID: 101750601[Serial]\n',
 '\n',
 '2. Author(s): Société de 

In [7]:
import numpy as np

# Make list of variables to keep
keep_vars = ['Author(s):', 'Title Abbreviation:', 'Title(s):', 
             'Publication Start Year:', 'Publication End Year:',
             'Frequency:', 'Country of Publication:', 'Language:',
             'ISSN:', 'NLM ID:']

# Make array of all journal information, filling in missings with 'None'
j_array = np.array(keep_vars).reshape(-1,10)
for file in j_files:
    f = open(file, 'r').readlines()

    # Extract relevant
    relevants = [i for i in f if any([j in i for j in keep_vars])]

    # Make dataframe
    i = 0
    output = []
    for var in keep_vars*200:
        if var in relevants[i]:
            output.append(relevants[i].split(':')[1])
            i += 1
            try:
                relevants[i]
            except:
                break
        else:
            output.append('None')
    j_array = np.concatenate([j_array, np.array(output).reshape(-1,10)])
    


In [34]:
# Drop '\n's
j_array = np.array(
    list(
        map(lambda x: [i.replace('\n', '') for i in x], j_array)
    )
)

# Author Table

In [1]:
import urllib.request

In [2]:
turl = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=30513499'
'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id=30513499&retmode=json.'

urllib.request.urlretrieve(turl, 'turl.txt')

('turl.txt', <http.client.HTTPMessage at 0x10dfcafd0>)

In [134]:
def artsum(a_url):
    with open(a_url, 'r') as file:
        string = file.read().replace('\n', '')
    splitstring = ' '.join(string.split()).split('{')
    aulist = [i[10:].replace('"', '').replace("'",'').replace(' }, ', '').split(', affil str ') 
                for i in splitstring if ' name ml' in i]
    audict = {a[0]: a[1].split(';') for a in aulist}
    
    abstract = [a.split('abstract "')[1].split('", idnum')[0] 
                for a in splitstring if 'abstract' in a][0]
    
    
    return splitstring

artsum('turl.txt')        

['Pubmed-entry ::= ',
 ' pmid 30513499, medent ',
 ' em std ',
 ' year 2018, month 12, day 5, hour 6, minute 0 }, cit ',
 ' title ',
 ' name "Associations between symptoms of depression and anxiety and cortisol responses to and recovery from acute stress." }, authors ',
 ' names std ',
 ' ',
 ' name ml "Fiksdal A", affil str "Department of Psychology and Volen National Center for Complex Systems, Brandeis University, Waltham, MA, United States." }, ',
 ' name ml "Hanlin L", affil str "Department of Psychology and Volen National Center for Complex Systems, Brandeis University, Waltham, MA, United States." }, ',
 ' name ml "Kuras Y", affil str "Department of Psychology and Volen National Center for Complex Systems, Brandeis University, Waltham, MA, United States." }, ',
 ' name ml "Gianferante D", affil str "Department of Psychology and Volen National Center for Complex Systems, Brandeis University, Waltham, MA, United States." }, ',
 ' name ml "Chen X", affil str "Department of Psycholo

In [135]:
turl2 = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id=30513499&retmode=json.'

In [136]:
import json

In [138]:

webURL = urllib.request.urlopen(turl2)
data = webURL.read()
encoding = webURL.info().get_content_charset('utf-8')
json.loads(data.decode(encoding))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)